In [10]:
using NamedArrays      # make sure you run Pkg.add("NamedArrays") first!

# import Stigler's data set
raw = readdlm("stigler.csv",',');
(m,n) = size(raw)

n_nutrients = 2:n      # columns containing nutrients
n_foods = 3:m          # rows containing food names

nutrients = raw[1,n_nutrients][:]   # the list of nutrients (convert to 1-D array)
foods = raw[n_foods,1][:]           # the list of foods (convert to 1-D array)

# lower[i] is the minimum daily requirement of nutrient i.
lower = Dict( zip(nutrients,raw[2,n_nutrients]) )

# data[f,i] is the amount of nutrient i contained in food f.
data = NamedArray( raw[n_foods,n_nutrients], (foods,nutrients), ("foods","nutrients") );

#Foods that are not vegan or gluten-free
bannedFoods = []
#Add banned foods to the list
for f in 1:4
    push!(bannedFoods, foods[f])
end

for f in 9:40
    push!(bannedFoods, foods[f])
end

#----Model----#
using JuMP
m = Model()

#Amount of daily food types in the diet
@defVar(m, diet[foods] >= 0)

#Total amount of daily nutrient n in the diet
@defExpr(totNutrient[n in nutrients], sum{data[f, n]*diet[f], f in foods})

#Meet RDA
@addConstraint(m, meetRDA[n in nutrients], totNutrient[n] >= lower[n])
#No non-gluten-free and no-non-vegan food in the diet
@addConstraint(m, glutenFreeVeganOnly[b in bannedFoods], diet[b] == 0)

#Minimize annual diet cost
@setObjective(m, Min, sum{diet[f]*1*365, f in foods})

solve(m)
println("Annual vegan and gluten-free diet cost: ", getObjectiveValue(m), " dollars")
println()
println("Daily diet composition (in dollar unit)")
println(getValue(diet))

Annual vegan and gluten-free diet cost: 45.58854783427841 dollars

Daily diet composition (in dollar unit)
diet: 1 dimensions, 77 entries:
 [                 Apples] = 0.0
 [        Asparagus (can)] = 0.0
 [                  Bacon] = 0.0
 [                Bananas] = 0.0
 [                 Butter] = 0.0
 [                Cabbage] = 0.011313245088275924
 [                Carrots] = 0.0
 [                 Celery] = 0.0
 [       Cheese (Cheddar)] = 0.0
 [              Chocolate] = 0.0
 [            Chuck Roast] = 0.0
 [                  Cocoa] = 0.0
 [                 Coffee] = 0.0
 [             Corn (can)] = 0.0
 [            Corn Flakes] = 0.0
 [              Corn Meal] = 0.005344246335991793
 [             Corn Syrup] = 0.0
 [                  Cream] = 0.0
 [                 Crisco] = 0.0
 [                   Eggs] = 0.0
 [  Evaporated Milk (can)] = 0.0
 [            Green Beans] = 0.0
 [      Green Beans (can)] = 0.0
 [            Ham, smoked] = 0.0
 [           Hominy Grits] = 0.0
 [